In [1]:
# Import dependencies
from sleeperpy import User, Leagues
import pandas as pd

In [2]:
# User inputs username and selects a league from landing page
username = "dirtywizard"
week_start = 0
week_end = 17
# Get User's Sleeper ID and Sleeper leagues
user_id = User.get_user(username)['user_id']
leagues = Leagues.get_all_leagues(user_id, 'nfl', 2024)
league_name_list = [x['name'] for x in leagues]
# Show the user the list of names and have them select one
print(league_name_list)
league_name = "No Coffee Sharps D.F.F.L."
# Find the league id for the selected league
league_id = [x['league_id'] for x in leagues if x['name'] == league_name][0]

['No Coffee Sharps D.F.F.L.']


In [3]:
# Receive user ID & league ID in GET request, then fetch league data from Sleeper API
selected_league = Leagues.get_league(league_id)
roster_positions = selected_league['roster_positions']
scoring_settings = selected_league['scoring_settings']
# Get the rosters for the league
rosters = Leagues.get_rosters(league_id)
users = Leagues.get_users(league_id)

In [10]:
# Bring in the franchise_id for each roster
roster_df = pd.DataFrame(rosters)
roster_df = roster_df[['players', 'owner_id']]
# Explode the roster_df to get a row for each player on each roster
roster_df = roster_df.explode('players')
roster_df['players'] = roster_df['players'].astype(str)
roster_df = roster_df.rename(columns={'players': 'id_sleeper'})
roster_df.head()

,id_sleeper,owner_id
0,11575,208058775418961920
0,11643,208058775418961920
0,8139,208058775418961920
0,10871,208058775418961920
0,10235,208058775418961920


In [11]:
# Get info about each franchise
franchises = pd.DataFrame()
franchises['owner_id'] = [x['user_id'] for x in users]
franchises['owner_name'] = [x['display_name'] for x in users]
franchises['team_name'] = [x['metadata']['team_name'] if 'team_name' in x['metadata'] else x['display_name'] for x in users]
# Create a True/False column for whether the owner is the user
franchises['is_user'] = False
franchises.loc[franchises['owner_id'] == user_id, 'is_user'] = True
franchises

,owner_id,owner_name,team_name,is_user
0,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
1,342869377223385088,nrector,The Fort Worth Fremen,False
2,412722740429082624,jccarm,DangeRUSS Last Ride,False
3,452577139540094976,2014champ,2014champ,False
4,461323803130064896,cdulgar,Idk much about soccer,False
5,725907041763303424,dirtywizard,Croccity Body Snatchers,True
6,904381167934095360,mdwglobe,GusTheBus,False
7,904398822170636288,SpacemnSpiff,Saskatoon Squatches,False
8,904399374891855872,comebackking15,Comeback 👑’s,False
9,904400906127458304,TrulyUhle,Verdanks Vacqueros FFC,False


In [12]:
# Merge roster_df with franchises to get the owner_name and team_name for each player
roster_df = pd.merge(roster_df, franchises, on='owner_id')
roster_df.head()

,id_sleeper,owner_id,owner_name,team_name,is_user
0,11575,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
1,11643,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
2,8139,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
3,10871,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False
4,10235,208058775418961920,ethrec,📜 Providence 🪬 Spirits 🥂,False


In [15]:
# Read RDS database 
# Select id_sleeper, full_name, age, team, number, position from players where id_sleeper in rostered_players
allplayers = pd.read_csv('test_data/rds_players.csv', dtype={'id_sleeper': 'str'}) 

# Read RDS database
predictions = pd.read_csv('test_data/rds_predictions.csv', dtype={'id_sleeper': 'str'})
# Get a list of rostered players 
rostered_players = roster_df['id_sleeper'].tolist()
# Get a list of the user's players
user_players = roster_df.loc[roster_df['is_user'] == True, 'id_sleeper'].tolist()
# Filter the predictions to only include players on the user's roster or on waivers
predictions = predictions.loc[(~predictions['id_sleeper'].isin(rostered_players)) | (predictions['id_sleeper'].isin(user_players))]
predictions = predictions.merge(allplayers[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position']], on='id_sleeper', how='left')

# Multiply each prediction column by the corresponding scoring setting
for column in predictions.columns:
    if column in scoring_settings:
        predictions[column] = predictions[column] * scoring_settings[column]
# Sum up the total points for each column in the scoring settings
predictions['predicted_fantasy_points'] = predictions[list(scoring_settings.keys())].sum(axis=1)
predictions = predictions.drop(columns=list(scoring_settings.keys())).drop(columns=['index_predictions'])
predictions.sort_values(by='predicted_fantasy_points', ascending=False, inplace=True, ignore_index=True)
predictions = predictions[['id_sleeper', 'full_name', 'age', 'team', 'number', 'position', 'week_of_season', 'predicted_fantasy_points']]
predictions.head()

,id_sleeper,full_name,age,team,number,position,week_of_season,predicted_fantasy_points
0,6904,Jalen Hurts,25.0,PHI,1.0,QB,1,23.41
1,6904,Jalen Hurts,25.0,PHI,1.0,QB,14,23.33
2,6904,Jalen Hurts,25.0,PHI,1.0,QB,11,22.97
3,6904,Jalen Hurts,25.0,PHI,1.0,QB,2,22.89
4,6904,Jalen Hurts,25.0,PHI,1.0,QB,15,22.79


In [16]:
# Merge the predictions with the roster_df to get the owner_name and team_name for each player
complete = pd.merge(predictions, roster_df, on='id_sleeper', how='left')
# Mark the unrostered players as 'Free Agents'
complete.loc[complete['owner_id'].isnull(), 'team_name'] = 'Free Agents'
complete.loc[complete['owner_id'].isnull(), 'owner_name'] = 'Free Agents'
complete.head()

,id_sleeper,full_name,age,team,number,position,week_of_season,predicted_fantasy_points,owner_id,owner_name,team_name,is_user
0,6904,Jalen Hurts,25.0,PHI,1.0,QB,1,23.41,725907041763303424,dirtywizard,Croccity Body Snatchers,True
1,6904,Jalen Hurts,25.0,PHI,1.0,QB,14,23.33,725907041763303424,dirtywizard,Croccity Body Snatchers,True
2,6904,Jalen Hurts,25.0,PHI,1.0,QB,11,22.97,725907041763303424,dirtywizard,Croccity Body Snatchers,True
3,6904,Jalen Hurts,25.0,PHI,1.0,QB,2,22.89,725907041763303424,dirtywizard,Croccity Body Snatchers,True
4,6904,Jalen Hurts,25.0,PHI,1.0,QB,15,22.79,725907041763303424,dirtywizard,Croccity Body Snatchers,True


In [17]:
# Convert the dataframe to json
waiverwire_json = complete.to_json(orient='records')
# Write to file
complete.to_json('test_data/waiverwire.json', orient='records')
